In [1]:
import os

In [3]:
DATA_DIR = os.path.join(os.getcwd(), 'RSNA_COVID_png_512')

TRAIN_DIR = os.path.join(DATA_DIR, 'train')


['Atypical', 'Indeterminate', 'Negative', 'Typical']


In [10]:
for classes in os.listdir(TRAIN_DIR):
    len_class = len(os.listdir(os.path.join(TRAIN_DIR, classes)))
    print(f'{classes}의 비율: ', round(len_class / 4842, 2))

Atypical의 비율:  0.08
Indeterminate의 비율:  0.17
Negative의 비율:  0.28
Typical의 비율:  0.47


In [7]:
379 + 839 +  1340 +  2284

4842

# 우리 모델의 문제

avgpool과 fc 레이어가 중첩되었음.  
그렇기 때문에 output의 channel이 1000이 나옴

![img](/home/pung/repo/Pytorch_MLOps/imgs/model.png)



```
self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  
self.fc = nn.Linear(512 * 4, num_classes)
```

In [14]:
import torch.nn as nn
import torch.nn.functional as F
from efficientnet_pytorch import EfficientNet
from torchvision import models

class Pre_Resnet50(nn.Module):
    def __init__(self, img_channel, num_classes, pretrained=True):
        super(Pre_Resnet50, self).__init__()
        self.backbone = models.resnet50(pretrained=pretrained)
        self.backbone.conv1 = nn.Conv2d(img_channel, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)

        ##### 변경 #########
        self.backbone.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.backbone.fc = nn.Linear(512 * 4, num_classes)
        
    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)
        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.backbone.fc(x)
        return x

if __name__ == '__main__':
    net = Pre_Resnet50(img_channel=1, num_classes=4)
    print(net)


Pre_Resnet50(
  (backbone): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
    

# 수정사항

In [12]:
import torch.nn as nn
import torch.nn.functional as F
from efficientnet_pytorch import EfficientNet
from torchvision import models

class Pre_Resnet50(nn.Module):
    def __init__(self, img_channel, num_classes, pretrained=True):
        super(Pre_Resnet50, self).__init__()
        self.backbone = models.resnet50(pretrained=pretrained)
        self.backbone.conv1 = nn.Conv2d(img_channel, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
        ##### 변경 #########
        self.backbone.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.backbone.fc = nn.Linear(512 * 4, num_classes)
    
    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)
        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.backbone.fc(x)
        return x

if __name__ == '__main__':
    net = Pre_Resnet50(img_channel=1, num_classes=4)
    print(net)

Pre_Resnet50(
  (backbone): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
    